w_01 | `requests`
====

Este workshop tiene como objetivo dar a conocer el funcionamiento básico de `requests` como herramienta principal para interactuar con la web.

In [ ]:
from pprint import pprint

import requests
from bs4 import BeautifulSoup

El uso más común que un científico de datos le va a dar a `requests` es interactuar con APIs para descargar datos. Esto implica usar la función `get` del paquete, que como dice su nombre, envía un GET a una URL que le indiquemos.

El siguiente bloque de código consulta una URL de prueba para que devuelva todos los posts disponibles (imaginar que es un foro en donde varios usuarios pueden crear artículo). Luego consultamos el `status_code` del request para confirmar que la URL funciona ([ver status posibles](https://developer.mozilla.org/es/docs/Web/HTTP/Status)) y decodificamos el contenido a formato JSON.

In [ ]:
r = requests.get("https://jsonplaceholder.typicode.com/posts")
print(r.status_code, "\n")
print(r.headers, "\n")
pprint(r.json()[:5])

En el bloque anterior se consultó el JSON directamente porque esa web devuelve resultados en JSON. Si no fuera así, en lugar de llamar al método `json` habría que ver los atributos `content` (si el contenido fuera binario) o `text` (en el resto de los casos). Veamos un caso de `text` con un informe epidemiológico del SINAE, para el cual usamos brevemente `BeautifulSoup` solo para hacer print del HTML prolijamente.

In [ ]:
r = requests.get("https://www.presidencia.gub.uy/comunicacion/comunicacionnoticias/informe-sinae-19-abril-2021")
print(r.status_code)
soup = BeautifulSoup(r.text)
print(soup.prettify())

`requests` puede emular todos los protocolos de HTTP (POST, PUT, DELETE, HEAD, etc.). El siguiente bloque crea un nuevo artículo mediante la función `post`.

Dado que se está creando un objeto, es necesario pasar algún tipo de data que debe estar en el formato que el sitio web espera.

In [ ]:
r = requests.post("https://jsonplaceholder.typicode.com/posts", data={"title": "foo", "body": "bar", "userId": 1})
pprint(r.json())

Podemos construir la URL de consulta directamente en `get`, lo que abre la puerta a oportunidades de automatización.

In [ ]:
payload = {"c": "213,223,298,", "s": "NGDPDPC,", "sy": "2000", "ey": "2020", "ssm": "0", "scsm": "1", "scc": "0", "ssd": "1", "ssc": "0", "sic": "0", "sort": "country", "ds": ".", "br": "1"}
r = requests.get("https://www.imf.org/en/Publications/WEO/weo-database/2020/October/weo-report?", params=payload)
r.url

Podríamos consultar varias URLs secuencialmente y guardar las respuestas. En ese caso es buena práctica usar un objeto `Session`, que reutiliza la conexión y reduce el tiempo de ejecución. Además, la información que necesitemos proveerle al sitio web puede ser establecida en la sesión una sola vez, en lugar de en cada request (por ejemplo, usuario y contraseña).

In [ ]:
s = requests.Session()
urls = ["https://www.cartelera.com.uy/averespectaculo.aspx?3305", 
        "https://www.cartelera.com.uy/averespectaculo.aspx?13766",
        "https://www.cartelera.com.uy/averespectaculo.aspx?3129"]
responses = []
for url in urls:
    r = s.get(url)
    responses.append(r.text)

responses[1]

La respuesta del servidor puede ser un archivo, y usamos `requests` para descargarlo. Por ejemplo, descarguemos el PDF de proyectos de inversión promovidos por el gobierno en 2020 desde la página del Ministerio de Economía.

In [ ]:
url_2020 = "https://www.gub.uy/ministerio-economia-finanzas/sites/ministerio-economia-finanzas/files/2021-01/comap_setiembre-2020.pdf"
r_2020 = requests.get(url_2020)
r_2020.status_code

Recordar que `get()` devuelve un `Response` object. En este caso, este objeto es un PDF. Para descargarlo vamos a "escribir" su contenido en un archivo usando un context manager. Tenemos que elegir el modo "wb" ya que vamos a escribir (**w**rite) un archivo **b**inario.

In [ ]:
with open("2020.pdf", "wb") as f:
    f.write(r_2020.content)